In [1]:
import sys
sys.path.append('../..')
from src.models import *

%run -i 'train_MLP.py'

[<tf.Tensor 'input_1_1:0' shape=(None, 1) dtype=int16>, <tf.Tensor 'input_2_1:0' shape=(None, 1) dtype=int16>]


TypeError: in converted code:

    ../../src/models/model_MLP.py:30 call  *
        user = self.user_input(x[0])
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/autograph/impl/api.py:427 converted_call
        f in m.__dict__.values() for m in (collections, pdb, copy, inspect, re)):
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/autograph/impl/api.py:427 <genexpr>
        f in m.__dict__.values() for m in (collections, pdb, copy, inspect, re)):
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_ops.py:1336 tensor_equals
        return gen_math_ops.equal(self, other)
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/gen_math_ops.py:3627 equal
        name=name)
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/op_def_library.py:536 _apply_op_helper
        repr(values), type(values).__name__, err))

    TypeError: Expected float32 passed to parameter 'y' of op 'Equal', got 'collect

In [2]:
import sys
sys.path.append('../..')
from src.models import *

%run -i 'train_GMF.py'

[10000   412]
INPUT
Tensor("input_1_2:0", shape=(None, 412), dtype=float32)
Train on 8000 samples, validate on 2000 samples
Epoch 1/50
INPUT
Tensor("IteratorGetNext:0", shape=(100, 412), dtype=float32)
INPUT
Tensor("IteratorGetNext:0", shape=(100, 412), dtype=float32)
4400/8000 [===============>..............] - ETA: 2s - loss: 33.4500

KeyboardInterrupt: 

# MLP

In [8]:
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow import keras
import pandas as pd
from sklearn.metrics import mean_absolute_error

dataset = pd.read_csv('../../data/Jester-Dataset-ratings.csv', delimiter=',')
dataset = dataset[:20000]

# zabezpecenie aby ID boli v rozmedzi (0, pocet_userov/jokov)
dataset['USER_ID'] = dataset['USER_ID'].astype('category').cat.codes.values
dataset['JOKE_ID'] = dataset['JOKE_ID'].astype('category').cat.codes.values


# rozdelenie datasetu
train, test = train_test_split(dataset, test_size=0.2)
y_true = test['Rating']

number_users = len(dataset['USER_ID'].unique())
number_jokes = len(dataset['JOKE_ID'].unique())


# Debug
print(train.shape)
print(test.shape)
print(y_true.shape)
print(number_users)
print(number_jokes)

# Nahodne vymyslene cislo, predstavuju shape vystupu z embedding vrstiev

emb_output_dim = 5 


# embeddingy sa vytvoria zvlast pre joke a usera a potom sa concatnu

joke_input = keras.layers.Input(shape=[1])
joke_emb = keras.layers.Embedding(number_jokes + 1, emb_output_dim)(joke_input)
joke_vector = keras.layers.Flatten()(joke_emb)

user_input = keras.layers.Input(shape=[1])
user_emb = keras.layers.Embedding(number_users + 1, emb_output_dim)(user_input)
user_vector = keras.layers.Flatten()(user_emb)

concat = keras.layers.concatenate([joke_vector, user_vector])

# skryte vsrtvy
dense = keras.layers.Dense(units=32, activation='relu')(concat)
dense_2 = keras.layers.Dense(units=16, activation='relu')(dense)
dense_3 = keras.layers.Dense(units=12, activation='relu')(dense_2)
output = keras.layers.Dense(units=1, activation='relu')(dense_3)


model = keras.Model([user_input, joke_input], output)

model.compile(
    optimizer='adam',
    loss= 'mean_absolute_error'
)

model.fit(
    [np.array(train['USER_ID']), np.array(train['JOKE_ID'])], 
    np.array(train['Rating']), 
    epochs=10, 
    verbose=1, 
    validation_split=0.1
)


# Vyhodnotenie

print(model.summary())

from sklearn.metrics import mean_absolute_error
y_pred = model.predict([np.array(test['USER_ID']), np.array(test['JOKE_ID'])])


tmp = pd.DataFrame()
tmp['user_id'] = test['USER_ID']
tmp['joke_id'] = test['JOKE_ID']
tmp['predicted_rating'] = y_pred
tmp['true_rating'] = y_true
tmp

(16000, 3)
(4000, 3)
(4000,)
466
140
Train on 14400 samples, validate on 1600 samples
Epoch 1/10
14400/14400 [==============================] - 1s 73us/sample - loss: 4.4211 - val_loss: 3.8821
Epoch 2/10
14400/14400 [==============================] - 1s 43us/sample - loss: 3.8265 - val_loss: 3.8281
Epoch 3/10
14400/14400 [==============================] - 1s 39us/sample - loss: 3.7787 - val_loss: 3.8081
Epoch 4/10
14400/14400 [==============================] - 1s 38us/sample - loss: 3.7616 - val_loss: 3.8262
Epoch 5/10
14400/14400 [==============================] - 1s 42us/sample - loss: 3.7468 - val_loss: 3.8350
Epoch 6/10
14400/14400 [==============================] - 1s 39us/sample - loss: 3.7345 - val_loss: 3.8165
Epoch 7/10
14400/14400 [==============================] - 1s 39us/sample - loss: 3.7225 - val_loss: 3.8603
Epoch 8/10
14400/14400 [==============================] - 1s 39us/sample - loss: 3.7106 - val_loss: 3.8140
Epoch 9/10
14400/14400 [==============================] - 

,user_id,joke_id,predicted_rating,true_rating
8911,206,57,2.467444,3.594
4185,96,61,8.527971,9.875
18290,424,4,0.000000,-0.375
8441,197,85,7.592752,9.469
3982,89,78,4.574152,7.938
...,...,...,...,...
9829,231,2,0.000000,6.688
4130,94,9,0.614550,4.656
12336,288,54,5.258519,-7.344
17549,407,11,0.862729,1.344


In [5]:
print(mean_absolute_error(y_true, y_pred))

3.8974805486772954
